# Softmax regression with batchnorm

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
%matplotlib inline
np.set_printoptions(threshold=np.nan, linewidth=140)

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True)

# 1.Forward prop for batchnorm

In [2]:
def one_hot_encoding(Yraw, K):
    H = np.arange(0, K).reshape(-1,1)
    Y = (H == Yraw).astype(np.int32)
    return Y

In [3]:
# dataset
K = 4
X = np.array([[1,1,0,0, 4,3,2,1, 7,6,5,4, 9,7,6,4],
              [8,6,4,3, 8,6,4,2, 6,5,4,3, 4,3,2,1]])
Y = np.array([[0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3]])



In [4]:
#visualization
colors = ['rgba(252, 0, 0, .8)', 'rgba(0, 200, 0, .8)', 'rgba(0, 0, 200, .8)', 'rgba(200, 0, 200, .8)']
dots = []
for k in range(K):
    x = X[0,k*4:(k+1)*4]
    y = X[1,k*4:(k+1)*4]
    trace = go.Scatter(x = x, 
                       y = y,
                       name = f'class: {k}',
                       mode = 'markers',
                       marker = { 'size'  : 10,
                                  'color' : colors[k]} )

    dots.append(trace)

layout = dict(title = 'Dataset visualisation',
#               yaxis = dict(zeroline = False),
#               xaxis = dict(zeroline = False)
             )

fig = dict(data=dots, layout=layout)
    
plotly.offline.iplot(fig)

In [5]:
Y = one_hot_encoding(Y, K)

In [6]:
# initialization
def initialize():
    m = X.shape[1]
    l = [ X.shape[0], Y.shape[0] ]   

    beta = np.zeros((l[1],1))
    gamma = np.ones((l[1],1))
    W = np.random.randn(l[1],l[0])*0.01

    Zmean = np.zeros((Y.shape[0], 1))
    Zstd  = np.zeros((Y.shape[0], 1))
    return W,gamma,beta,Zmean,Zstd

In [7]:
def batchnorm(Z, gamma, beta):
    m = Z.shape[1]
    mean = np.mean(Z,axis=1,keepdims=True)
    Z0 = Z - mean
    var = np.sum(np.square(Z0), axis=1, keepdims=True) / m 
    std = np.sqrt(var+epsilon)
    
    Znorm = Z0 / std
    Zbn = Znorm * gamma + beta
    
    return Zbn, mean, Z0, var, std, Znorm


def softmax(Z):
    D = -np.max(Z, axis=0) #try to remove it if there will be a problem
    exponents = np.exp(Z + D)
    expsum = np.sum(exponents ,axis=0, keepdims=True)
    return exponents / expsum


def cross_entropy(Y, H):
    Y = Y.astype(np.bool)
    Yhj = H[Y]
    Losses = -np.log(Yhj[np.nonzero(Yhj)])
    return np.mean(Losses)

In [8]:
# forward
def forward(X,Y, W, gamma, beta):
    Z = W @ X 
    Zbn, Zmean, Z0, Zvar, Zstd, Znorm = batchnorm(Z, gamma, beta)
    A = softmax(Zbn)
    cost = cross_entropy(Y,A)
    cache = {'Z'    : Z, 
             'Znorm': Znorm,
             'Zbn'  : Zbn, 
             'Zmean': Zmean, 
             'Z0'   : Z0, 
             'Zvar' : Zvar, 
             'Zstd' : Zstd,
             'A'    : A,
             'cost' : cost}
    return cache    


In [9]:
# test mode
def test_batchnorm(Z, gamma, beta, Zmean, Zstd):   
    Znorm = (Z - Zmean) / Zstd    
    Zbn = Znorm * gamma + beta    
    return Zbn


def test_forward(X, W, gamma, beta, Zmean, Zstd):
    Z = W @ X 
    Zbn = test_batchnorm(Z, gamma, beta, Zmean, Zstd)
    A = softmax(Zbn)
    return A


def predict(X, W, gamma, beta, Zmean, Zstd):
    A = test_forward(X,W, gamma, beta, Zmean, Zstd)
    return np.argmax(A, axis=0)   

# 2. Derivative of batchnorm layer

After long calculations, we have these formulas.



$$\begin{align*}\partial \beta_{n}  & = \sum_{i=0}^{m}Zbn_{n,i} \\
\partial \gamma_{n} & = \sum_{i=0}^{m}\left(\partial Zbn_{n,i} \odot Znorm\right)\\
\partial Znorm & = \partial Zbn \odot \gamma \\
\end{align*}$$

**Let:**

$\mu = \frac{1}{m}\sum_{i=0}^{m}Z_{n,i};$

$Z_{\varnothing} = Z\circleddash \mu;$

$var = \frac{1}{m}\sum_{i=0}^{m}\left(Z_{\varnothing}^{2}\right)_{n,i};$

$std = \sqrt{var + \epsilon};$




$$\begin{align*}
\partial Z_{\varnothing} & = \left(\partial Znorm ~ \odot \frac{1}{std}\right)  + \left( \sum_{i=0}^{m} \left( \partial Znorm \odot Z_{\varnothing} \right)_{n,i}  \odot \frac{-1}{var + \epsilon} \odot \frac{1}{2 \cdot std} \odot \frac{\boldsymbol{J}}{m} \odot \left(2 \cdot Z_{\varnothing} \right) \right) \\
\partial Z & = \partial Z_{\varnothing} \circleddash\frac{1}{m} \cdot \sum_{i=0}^{m}\partial Z_{\varnothing\left(n,i\right)}
\end{align*}$$ 

**Where:**

$Z,~Z_{\varnothing},~\partial Z,~\partial Znorm  \in \mathbb{R}^{n,m},$

$\mu,~var,~std  \in \mathbb{R}^{n,1},$

$\gamma, \beta \in \mathbb{R}^{n,1},$

$\boldsymbol{J} \in \begin{bmatrix}
        1 &      1  &  \dots  &       1 \\
        1 &      1  &  \dots  &       1 \\
   \vdots & \vdots  &  \ddots &  \vdots \\
        1 &      1  &  \dots  &       1 \\
\end{bmatrix}^{(n,m)}$

I used signs $\odot, \circleddash$ to emphasize element-wise operation with possible broadcasting.



In [10]:
def backward(X,Y, gamma, beta, cache):
    A     = cache['A']
    Z     = cache['Z']
    Znorm = cache['Znorm']
    Zstd  = cache['Zstd']
    Zvar  = cache['Zstd']
    Zmean = cache['Zmean']
    Z0    = cache['Z0']
    J     = np.ones(Z.shape)
    m     = X.shape[1]    
    
    dZbn = A - Y
    dbeta = np.sum(dZbn, axis=1, keepdims=True)
    dgamma = np.sum(dZbn * Znorm, axis=1, keepdims=True)
    dZnorm = dZbn * gamma
    dZ0 = (dZnorm/Zstd) + Z0*(-np.sum(dZnorm*Z0, axis=1, keepdims=True) / (m*(Zvar+epsilon)*Zstd))
#     dZ0 = (dZnorm/Zstd) + (np.sum(dZnorm*Z0, axis=1, keepdims=True)
#                            *(-np.reciprocal(Zvar+epsilon))
#                            *np.reciprocal(2*Zstd)
#                            *(J/m)
#                            *(2*Z0))
    dZ = dZ0 - np.sum(dZ0, axis=1, keepdims=True) / m
    dW = dZ @ X.T / m
    
    return dW, dgamma, dbeta

In [11]:
def train(X, Y, W, gamma, beta, Zmean, Zstd, learning_rate=0.01, iterations=1000, report=50):
    for i in range(iterations):
        cache = forward(X,Y,W,gamma,beta)
        dW, dgamma, dbeta = backward(X,Y, gamma, beta, cache)
        W -= learning_rate*dW
        gamma -= learning_rate*dgamma
        beta -= learning_rate*dbeta

        Zmean = 0.7*Zmean + 0.3*cache['Zmean']
        Zstd  = 0.7*Zstd  + 0.3*cache['Zstd']
        if not i%report:
            print(f'[{i:>06}] Cost: {cache["cost"]:.7f}')
    return W, gamma, beta, Zmean, Zstd

In [17]:
epsilon = 1e-8
W,gamma,beta,Zmean,Zstd = initialize()
W, gamma, beta, Zmean, Zstd = train(X,Y,W,gamma,beta,Zmean,Zstd, learning_rate=1, iterations=1000, report=200)

[000000] Cost: 1.7377228
[000200] Cost: 0.0497112
[000400] Cost: 0.0180360
[000600] Cost: 0.0063595
[000800] Cost: 0.0030831


In [18]:
x1, x2 = np.meshgrid(np.arange(0, 10, 0.1),
                     np.arange(0, 10, 0.1))
dp = np.row_stack((x1.ravel(),x2.ravel()))

prediction = predict(dp, W, gamma, beta, Zmean, Zstd)

prediction = prediction.reshape(100,100)


cntr =  {'z': prediction,
        'colorscale':'Jet',
        'type': u'contour',
        'dx': 0.1,
        'x0': 0,
        'dy': 0.1,
        'y0': 0 }

data = go.Data([go.Contour(cntr)])

data = data + dots

layout = dict(title = 'Decision surfaces visualisation')

fig = dict(data=data, layout=layout)
    
plotly.offline.iplot(fig)